In [1]:
%run bert.ipynb

attention pattern raw MATCH!!!!!!!!
 SHAPE (2, 12, 3, 3) MEAN: 0.006603 STD: 0.1129 VALS [0.05867 0.01584 -0.05108 -0.212 -0.2484 0.05948 0.04164 0.0499 -0.05705 0.1175...]
attention MATCH!!!!!!!!
 SHAPE (2, 3, 768) MEAN: 0.0001962 STD: 0.1178 VALS [-0.1035 0.03562 -0.07408 0.1086 0.01748 0.05069 -0.14 -0.01052 0.01697 0.04285...]
bert MATCH!!!!!!!!
 SHAPE (2, 3, 768) MEAN: -0.001554 STD: 0.1736 VALS [-0.08316 -0.09165 -0.03188 -0.03013 0.1001 0.09549 -0.1046 0.07742 0.0424 0.05553...]
bert mlp MATCH!!!!!!!!
 SHAPE (2, 3, 768) MEAN: -0.0001934 STD: 0.1044 VALS [-0.1153 0.1189 -0.0813 0.1021 0.0296 0.06182 0.0341 0.1446 0.2622 -0.08507...]
layer norm MATCH!!!!!!!!
 SHAPE (20, 10) MEAN: -1.431e-08 STD: 1.003 VALS [0.6906 -0.84 1.881 1.711 -0.5117 -0.9577 -0.1387 -0.6943 -0.6741 -0.4662...]
bert MATCH!!!!!!!!
 SHAPE (2, 3, 768) MEAN: -4.139e-09 STD: 1 VALS [0.007131 -0.04372 0.6502 -0.5972 -1.097 0.7267 0.1275 -0.6035 -0.2226 0.2145...]
{'input_ids': [[101, 8667, 117, 146, 1821, 170, 5650

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


AssertionError: error in comparing Berts
 SHAPE (10, 20, 28996) MEAN: -2.732 STD: 2.414 VALS [-5.65 -6.041 -6.096 -6.062 -5.945 -5.777 -5.977 -6.015 -6.028 -5.935...] 
!=
 SHAPE (10, 20, 28996) MEAN: -2.732 STD: 2.413 VALS [-5.65 -6.041 -6.096 -6.062 -5.946 -5.777 -5.977 -6.015 -6.028 -5.935...]

In [2]:
import transformers
tokenizer = transformers.AutoTokenizer.from_pretrained("bert-base-cased")

In [4]:
def make_bert():
    my_bert = Bert(
        vocab_size=28996, hidden_size=768, max_position_embeddings=512, 
        type_vocab_size=2, dropout=0.1, intermediate_size=3072, 
        num_heads=12, num_layers=12
    )
    pretrained_state_dict = pretrained_bert.state_dict()

    #pretrained_keys = pretrained_state_dict.keys()
    #my_keys = my_bert.state_dict().keys()
    #print(set(my_keys) - set(pretrained_keys))

    del pretrained_state_dict["classification_head.weight"]
    del pretrained_state_dict["classification_head.bias"]

    my_bert.load_state_dict(pretrained_state_dict)
    return my_bert

In [79]:
def ascii_art_probs(sentence):
    bert = make_bert()
    token_ids = torch.Tensor(tokenizer.encode(sentence)).long().unsqueeze(0)
    mask_idxs = set()
    for idx, token_id in enumerate(token_ids[0]):
        if token_id == 103: # 103 == [MASK]
            mask_idxs.add(idx)
    unnormalized_output = bert(token_ids)
    output = torch.log_softmax(unnormalized_output, dim=-1)
    top_k = torch.topk(output, 5, dim=-1)
    results = []
    for seq_i, seq_top_k in enumerate(top_k.indices[0]):
        if seq_i in mask_idxs:
            results.append(list(zip(tokenizer.convert_ids_to_tokens(seq_top_k), [round(x.item(), 2) for x in top_k.values[0][seq_i].exp()])))
    for l in results:
        print(l)

ascii_art_probs("The fish loves to eat [MASK] and [MASK] and [MASK] and [MASK] and [MASK] and [MASK] and [MASK].")


[(',', 0.12), ('fish', 0.12), ('meat', 0.08), ('eggs', 0.05), ('food', 0.05)]
[('fish', 0.07), ('meat', 0.05), ('eggs', 0.03), ('food', 0.02), ('bread', 0.02)]
[('fish', 0.05), ('eggs', 0.04), ('meat', 0.04), ('fruit', 0.02), ('food', 0.02)]
[('fish', 0.04), ('meat', 0.02), ('salt', 0.01), ('wine', 0.01), ('food', 0.01)]
[('fish', 0.04), ('meat', 0.02), ('eggs', 0.02), ('birds', 0.02), ('food', 0.01)]
[('fish', 0.04), ('food', 0.02), ('meat', 0.02), ('birds', 0.01), ('water', 0.01)]
[('fish', 0.03), ('birds', 0.02), ('more', 0.02), ('honey', 0.01), ('food', 0.01)]


In [83]:
class ClassificationBERT():
    def __init__(self, num_classes):
        VOCAB_SIZE = 28996
        DROPOUT=0.1
        self.bert = Bert(
            vocab_size=VOCAB_SIZE, hidden_size=768, max_position_embeddings=512, 
            type_vocab_size=2, dropout=DROPOUT, intermediate_size=3072, 
            num_heads=12, num_layers=12
        )
        self.dropout = torch.nn.Dropout(DROPOUT)
        # TODO attach this to the last block of BERT as a hidden_size x num_classes linear layer,
        # not on top of the vocab_size output of BERT
        self.classification_layer = torch.nn.Linear(VOCAB_SIZE, num_classes)

        pretrained_state_dict = pretrained_bert.state_dict()
        del pretrained_state_dict["classification_head.weight"]
        del pretrained_state_dict["classification_head.bias"]
        self.bert.load_state_dict(pretrained_state_dict)

    def forward(self, X):
        return self.classification_layer(self.dropout(self.bert(X)))

In [97]:
import torchtext
import random
data_train, data_test = torchtext.datasets.IMDB(root='.data', split=('train', 'test'))
data_train_list = list(data_train)
random.shuffle(data_train_list)

In [129]:

def make_batches(data_train_list, batch_size, max_seq_len):
    batches = []
    for batch_start in range(0, len(data_train_list), batch_size):
        batch_end = min(batch_start + batch_size, len(data_train_list))
        ratings, sentences = [list(tup) for tup in list(zip(*data_train_list[batch_start:batch_end]))]
        tokens = tokenizer(sentences, padding='longest', max_length=max_seq_len, truncation=True)
        ratings01 = torch.tensor([ rating == 'pos' for rating in ratings ], dtype=torch.int8)
        tokens_tensor = torch.Tensor(tokens.input_ids)
        batches.append((tokens_tensor, ratings01))
    return batches

[(tensor([[  101.,   107., 12510.,   102.],
          [  101.,  1258.,  3195.,   102.],
          [  101.,   146.,  1821.,   102.],
          [  101.,  1409.,  1128.,   102.],
          [  101.,  1258.,   107.,   102.],
          [  101.,  1332.,  1122.,   102.],
          [  101., 22449.,   170.,   102.]]),
  tensor([0, 1, 1, 1, 0, 1, 0], dtype=torch.int8)),
 (tensor([[  101.,   146.,  1138.,   102.],
          [  101.,   146.,   112.,   102.],
          [  101.,  2048.,   117.,   102.],
          [  101.,  9498.,  1103.,   102.],
          [  101., 12107.,  5253.,   102.],
          [  101.,   178.,  8703.,   102.],
          [  101.,   146.,  1108.,   102.]]),
  tensor([0, 0, 1, 0, 1, 0, 0], dtype=torch.int8)),
 (tensor([[  101.,  1109., 22891.,   102.],
          [  101.,   178.,  2140.,   102.],
          [  101.,  2907.,  1227.,   102.],
          [  101., 13411., 10197.,   102.],
          [  101.,  1258., 10761.,   102.],
          [  101., 15783.,   161.,   102.],
          [ 